Ruggiero Seccia

La Sapienza University of Rome

Email: ruggiero.seccia@uniroma1.it

Phone: +39 3318606535

# Standard Nurses Rostering Problem (V3)


This notebook implements a version of the NRP problem in the case the number of nurses is not enough to guarantee the satisfaction of the working condition regulations within the ward. __We consider the possibility that each nurse are asked to work more than one shift per day but up to $H^{\max}$ hours per period__. To this aim, we assume that each nurse can work for a fraction  $(1-c)$ or $c$ of the shift respectively before or after their shift. E.g. by fixing $c=0.5$ we allow each nurse to work half shift more before or after their proper shift. This allow us to reduce the number of shifts with insufficient number of nurses while minimizing stress for the healthcare personnel by defining balanced schedules.



## importing the packages

In [67]:
import numpy as np
try:
    from docplex.mp.model import Model
except:
    !pip install docplex
from docplex.mp.model import Model

import pandas as pd

try:
    import ipywidgets
except:
    !pip install ipywidgets
from ipywidgets import interact
import ipywidgets as widgets
import copy

## Parameters specification
Let us consider a department in a hospital with a given number of nurses $N$. We want to organize their shifts for the next $T$ days, e.g. $T=7$ one week or $T=30$ next month,  and for all the followings so to minimize the effort required by the staff to satisfy the demand. By contract, each nurse $i$ has to work $H_i$ hours over the time horizon $T$ (e.g. each nurse must work at least 36 hours per week, $H=36$ and $T=7$). If the $i$th nurse works for a number of hours higher than $H_i$, then it is counted as extra work and then paid more by the healthcare structure. Each day three shifts need to be covered by the nurses: Morning, Afternoon, Night.

Each shift $s$ requires $R_s$ nurses and lasts $h_s$ hours. Each nurse cannot cover more than one shift per day. Moreover, we have the further constraint that if a nurse covers a night shift then they need to rest and cannot work the following day. 

Let us  consider the parameter $p_i$ which brings information about the previous period. Namely, $p_i$ is a boolean parameter such that 
\begin{equation*}
    p_i=
    \begin{cases}
    1  \qquad \text{if the } i \text{th nurse worked on the last day of the previous period} \\
    0 \qquad \text{otherwise.}
    \end{cases}
\end{equation*}

Each nurse can work more than one shift per day but up to $H^{\max}$ hours per period. In particular, each nurse is allowed to work an extra half shift before or after entering the assigned shift.


In [68]:
# number of nurses
N = 10
nurses = ['Nurse_' +str(n) for n in range(N)]
# periods to schedule
T = 7
days = ['Day_' +str(t) for t in range(T)]
# shifts
S = ['Morning', 'Afternoon', 'Night']


# standard number of hours by contract per nurse (e.g. 6 hours per day, 36 per week)
H_base = 36*np.floor(T/7) + 6*np.mod(T,7)
H = {n:j for n in nurses for j in [H_base]*len(nurses)}

# maximum number of hours a nurse can work including the extra shifts (e.g. 10 hours per day, 60 per week)
H_max = 60*np.floor(T/7) + 10*np.mod(T,7)
H_MAX = {n:j for n in nurses for j in [H_max]*len(nurses)}

# each nurse can work half of the shift before or aftering their proper shift
c = 0.5 

# update some nurses values
# H['Nurse_1'] =20

# number of nurses required per shift
R = {'Morning' : 5,
     'Afternoon' : 5,
     'Night' : 3}
# duration of each shift
h = {'Morning' : 7,
     'Afternoon' : 8,
     'Night' : 9}

# list of nurses that on the last day of the previous period covered  the night shift
p_list= ['Nurse_0']
# dictionary with the values of p per each nurse
p = {n:0 for n in nurses}
# update the dictionary with p_list
for pp in p_list:
    p[pp]=1


x_i_4_1 is the boolean array with the information about the first shift of the next period. It should be 0 (i.e. we don't know who will work on the first shift of the next period)
\begin{equation*}
    x_{i41}=
    \begin{cases}
    1  \qquad \text{if the } i \text{th nurse works on the first shift of the first day of the next period} \\
    0 \qquad \text{otherwise.}
    \end{cases}
\end{equation*}





In [69]:
x_i_4_T = [0]*N

## Optimization model 

To formulate this optimization problem, let us introduce the binary variable $x_{ist}\in\{0,1\}$ such that 
\begin{equation*}
    x_{ist}=
    \begin{cases}
    1  \qquad \text{if nurse } i \text{th covers shift } s \text{th on day } t\text{th} \\
    0 \qquad \text{otherwise}
    \end{cases}
\end{equation*}


 We introduce the two additional integer variables $z_{ist},q_{ist}\in\{0,1\}$ such that:
\begin{equation*}
    q_{ist}=
    \begin{cases}
    1  \qquad \text{if nurse } i \text{th works the last $ch_s$ hours of the  shift } s  \text{th on day } t\text{th} \\
    0 \qquad \text{otherwise.}
    \end{cases}
\end{equation*}
\begin{equation*}
    z_{ist}=
    \begin{cases}
    1  \qquad \text{if nurse } i \text{th works the first $ch_s$ hours of the  shift } s  \text{th on day } t\text{th} \\
    0 \qquad \text{otherwise.}
    \end{cases}
\end{equation*}

Finally, we define the variable $\alpha_{st} \in R $ which represents the number of nurses that are missing to satisfy the minimum demand of the $s$th shift on the $t$th department.


In [72]:
mdl = Model('Scheduling')

# create the variables
idx_x = [(i,s,t) for i in nurses for s in S for t in days]
x = mdl.binary_var_dict(idx_x)
z = mdl.binary_var_dict(idx_x)
q = mdl.binary_var_dict(idx_x)

idx_alpha = [(s,t) for s in S for t in days]
alpha = mdl.continuous_var_dict(idx_alpha)

### Objective function
We want to find the optimal schedule $x^\star,z^\star,q^\star$ such that the proper hours worked by each nurse plus the extra shift working hours is minimized while guaranteeing the satisfaction of the working condition regulations within the ward. Of course, we are also interested in minimizing the number of shifts that cannot be covered by the nurses ($\sum_{st}\alpha_st$)

$$
\sum_{i=1}^N\sum_{s=1}^3\sum_{t=1}^T  \left(h_s x_{ist} + \frac 1 2 h_s z_{ist} +\frac 1 2 h_s q_{ist} \right) +\rho \sum_{s=1}^3\sum_{t=1}^T \alpha_{st}
$$
  
with $\rho>\max_s h_s$. Note that, even if $\alpha$ represents a discrete quantity, it is modeled as a continuous variable since at the optimum it will achieve only integer values (thanks to some further constraints defined in the following).

In [73]:
mdl.minimize(mdl.sum(x[i,s,t]*h[s] +0.5*h[s]*z[i,s,t]+0.5*h[s]*q[i,s,t] for i in nurses for s in S for t in days)
             +(1e4*mdl.sum(alpha[s,t] for s in S for t in days)))

### Constraints
- Each person cannot cover more than one shift in the same day. 
$$ \sum_{s=1}^3x_{ist}\leq 1 \qquad \forall i,t$$
- The number of personnel per each shift in each day is satisfied. 
$$ \sum_{i=1}^N \left(x_{ist}+z_{ist}\right)+\alpha_{st} \geq R_{s} \qquad \forall s, t $$
In the following we require $ \sum_{i=1}^N z_{ist}=\sum_{i=1}^N q_{ist} \qquad \forall s,t$, that's why we can define this constraint in this way

In [75]:
mdl.add_constraints(mdl.sum(x[i,s,t] for s in S) <= 1 for i in nurses for t in days);
mdl.add_constraints(mdl.sum(x[i,s,t] + z[i,s,t] for i in nurses)+alpha[s,t]>= R[s]  for s in S for t in days);


- Each nurse can work a maximum number of hours $H^{\max}$ without burning out:
$$  \sum_{s=1}^3\sum_{t=1}^T h_s \left(x_{ist}+cz_{ist}+(1-c) q_{ist}\right) \le H^{\max}\qquad \forall\ i=1,...,N   $$ 
- Each nurse works at minimum the number of hours required by contract (excluded the extra hours). 
$$ \sum_{s=1}^3\sum_{t=1}^T x_{ist}h_s\geq H_i \qquad \forall i $$ 

In [76]:
mdl.add_constraints(mdl.sum(h[s]*(x[i,s,t]+c*z[i,s,t]+(1-c)*q[i,s,t]) for s in S for t in days) <= H_MAX[i] for i in nurses );
mdl.add_constraints(mdl.sum(x[i,s,t]*h[s] for s in S for t in days) >= H[i] for i in nurses );

- If a nurse covers a night shift, then the next day they cannot work;
$$ x_{i3t}+\sum_{s=1}^3 x_{ist+1}\leq 1 \qquad \forall i,t $$ 
- Each nurse cannot work on the first day of the new period if they worked on the last shift of the previous period. 
$$ \sum_{s=1}^3 x_{is1}\leq (1-p_i) \qquad \forall i  $$ 

In [77]:
mdl.add_constraints( x[i,S[-1],t] + mdl.sum(x[i,s,days[j+1]] for s in S )<= 1 for i in nurses for j,t in enumerate(days[:-1]) );
mdl.add_constraints(mdl.sum(x[i,s,days[0]] for s in S ) <= (1-p[i]) for i in nurses );

-  each nurse cannot work more than one extra shift per day.
\begin{equation*}
    \sum_{s=1}^3 \left(z_{ist}+ q_{ist}+ \right)\le 1  \quad \forall i=1,...,N \; t=1,...,T
\end{equation*} 
-  nurses cannot cover extra shifts if they have already been assigned to work on that shift:
$$ z_{ist}\leq 1-x_{ist} \qquad \forall i,s,t$$
$$ q_{ist}\le 1 - x_{ist} \qquad \forall i,s,t$$
- The number of nurses covering the first and the second hslf of each shift, have to be the same (this allow us to define the variable $\alpha_{st}$ as continuous and not as discrete)
$$ \sum_{i=1}^N z_{ist}=\sum_{i=1}^N q_{ist} \qquad \forall s,t$$


In [78]:
mdl.add_constraints(mdl.sum(z[i,s,t] +q[i,s,t] for s in S)<= 1 for i in nurses for t in days);
mdl.add_constraints(z[i,s,t]<=1-x[i,s,t] for i in nurses for s in S for t in days);
mdl.add_constraints(q[i,s,t]<=1-x[i,s,t] for i in nurses for s in S for t in days);
mdl.add_constraints(mdl.sum(z[i,s,t] for i in nurses)== mdl.sum(q[i,s,t] for i in nurses) for s in S for t in days);

-  the additional hours are joined to a proper shift
$$ z_{ist}\leq x_{is-1t} \qquad \forall i,s,t$$
$$ q_{ist}\leq x_{is+1t} \qquad \forall i,s,t$$
where $x_{i4t}=x_{i1t+1}$ and $x_{i0t} = x_{i3t-1}$. Moreover, $x_{i01}=p_i$ and $x_{i4T}$ is assumed to be zero (i.e. nobody works on the first day of the next period).

In [79]:
for idx_i,i in enumerate(nurses):
    for idx_s, s in enumerate(S):
        for idx_t,t in enumerate(days):
            if s == S[0] and t == 'Day_0':
                mdl.add_constraint(z[i,s,t]<=p[i])
            elif s == S[0]:
                mdl.add_constraint(z[i,s,t]<=x[i,S[-1],days[idx_t-1]])
            else:
                mdl.add_constraint(z[i,s,t]<=x[i,S[idx_s-1],t])

In [80]:
for idx_i,i in enumerate(nurses):
    for idx_s, s in enumerate(S):
        for idx_t,t in enumerate(days):
            # if it is the last day of the period T
            if s == S[-1] and t == 'Day_'+str(T-1):
                mdl.add_constraint(q[i,s,t]<=x_i_4_T[idx_i])

            elif s == S[-1]:
                mdl.add_constraint(q[i,s,t]<=x[i,S[0],days[idx_t+1]])
            else:
                mdl.add_constraint(q[i,s,t]<=x[i,S[idx_s+1],t])

- if a nurse covers as extra hours the first part of a night shift, then they cannot cover the morning shift of the next day, and similarly if they cover the second part of the night shift, then they cannot cover the afternoon shift of the previous day:
\begin{equation*}
    z_{i3t}+ x_{i1t+1}\leq 1 \qquad \forall i=1,...,N \forall t=1,...,T-1
\end{equation*}
\begin{equation*}
    q_{i3t}+ x_{i2t}\leq 1 \qquad \forall i=1,...,N \forall t=1,...,T
\end{equation*}


In [81]:
for i in nurses:
    for idx_t,t in enumerate(days):
        # if it is the last day of the period T
        if 'Day_'+str(T-1):
            mdl.add_constraint(z[i,S[-1],t]<=1)

        else:
            mdl.add_constraint(z[i,S[-1],t]-x[i,S[0],days[idx_t+1]]<=1)

In [82]:
for idx_i,i in enumerate(nurses):
    for idx_t,t in enumerate(days):
        mdl.add_constraint(z[i,S[-1],t]-x[i,S[-2],t]<=1)

### Additional constraints
- In order to avoid unbalanced solutions with some nurses covering too many hours compared to others, we might require both the number of extra hours worked and the overall number of hours worked by each nurse to be close enough to the mean according to some pre-specified paramter $\mathcal{K}_2$ and the overall number of hours worked by each nurse

\begin{equation*}
\left\vert \sum_{s=1}^3\sum_{t=1}^Tz_{ist}- \frac{\sum_{i=1}^N\sum_{s=1}^3\sum_{t=1}^Tz_{ist}}{N} \right\vert \leq \mathcal{K}_2  \qquad \forall i
\end{equation*}
\begin{equation*}
\left\vert \sum_{s=1}^3\sum_{t=1}^Tq_{ist}- \frac{\sum_{i=1}^N\sum_{s=1}^3\sum_{t=1}^Tq_{ist}}{N} \right\vert \leq \mathcal{K}_2  \qquad \forall i
\end{equation*}


\begin{equation*}
\left\vert\sum_{s,t} \left(x_{ist}h_s+\bar w\left( z_{ist}+q_{ist}\right)\right) - \frac{\sum_{i,s,t}\left(x_{ist}h_s+\bar w\left( z_{ist}+q_{ist}\right)\right)}{N} \right\vert\leq  \mathcal{K}_3\qquad \forall i
\end{equation*}


### Defining KPI

In [83]:
mdl.add_kpi(mdl.sum(R[s]- mdl.sum(x[i,s,t] for i in nurses)>=1 for s in S for t in days), '# of shifts not completely covered');
mdl.add_kpi(mdl.max(alpha[s,t] for s in S for t in days), 'Maximum # missing nurses in one shift')
mdl.add_kpi(mdl.sum(alpha[s,t] for s in S for t in days), 'Sum of missing nurses in all the shifts')

mdl.add_kpi(mdl.max(mdl.sum(x[i,s,t]*h[s] for s in S for t in days)for i in nurses), 'Maximum # hours worked')
mdl.add_kpi(mdl.min(mdl.sum(x[i,s,t]*h[s] for s in S for t in days)for i in nurses), 'Minimum # hours worked');

DecisionKPI(name=Minimum # hours worked,expr=min(7x1+7x2+7x3+7x4+7x5+7x6+7x7+8x8+8x9+8x10+8x11+8x12+8x13+8x14..)

### Solve the problem

In [84]:
# We set a limit time for computations
mdl.set_time_limit(240)

In [85]:
mdl.print_information()
mdl.solve(log_output = False)
mdl.solution.solve_details

Model: Scheduling
 - number of variables: 692
   - binary=651, integer=0, continuous=41
 - number of constraints: 1293
   - linear=1272, equiv=21
 - parameters:
     parameters.timelimit = 240.00000000000000
 - problem type is: MILP


docplex.mp.SolveDetails(time=5.89,status='integer optimal solution')

In [89]:
mdl.report()

* model Scheduling solved with objective = 130597.000
*  KPI: # of shifts not completely covered      = 14.000
*  KPI: Maximum # missing nurses in one shift   = 3.000
*  KPI: Sum of missing nurses in all the shifts = 13.000
*  KPI: Maximum # hours worked                  = 52.000
*  KPI: Minimum # hours worked                  = 40.000


In [86]:
status = int('optimal' in mdl.solve_details.status)
status

1

## Analysing the solution
Below we provide simple tools to analyse the solution obtained

In [90]:
def sol_to_pandas(x):
    '''
    takes in input the solution of the optimization problem as a dictionary 
    returns the solution as a dataframe 
    '''
    sol = pd.DataFrame(columns = ['Nurse', 'Shift', 'Day'])
    k = 0
    for key, value in x.items():
        if value>0:
            sol.loc[k] =np.array([i for i in key])
            k+=1
    return sol

In [91]:
def alpha_to_pandas(x):
    '''
    takes in input the solution of the optimization problem as a dictionary 
    returns the solution as a dataframe 
    '''
    sol = pd.DataFrame(columns = ['Shift', 'Day', 'value'])

    k=0
    for key, value in alpha_star_dict.items():
        if value>0:
            sol.loc[k] =np.array([i for i in key]+[value])
            k+=1
        
    return sol

In [92]:
# transform the solution into a dataframe
x_star_dict =mdl.solution.get_value_dict(x)
sol_x = sol_to_pandas(x_star_dict)

z_star_dict =mdl.solution.get_value_dict(z)
sol_z = sol_to_pandas(z_star_dict)

q_star_dict =mdl.solution.get_value_dict(q)
sol_q = sol_to_pandas(q_star_dict)

alpha_star_dict = mdl.solution.get_value_dict(alpha)
sol_alpha = alpha_to_pandas(alpha_star_dict)

In [93]:
print(sol_x['Nurse'].value_counts())


Nurse_3    7
Nurse_1    7
Nurse_2    7
Nurse_5    7
Nurse_9    7
Nurse_7    7
Nurse_8    6
Nurse_6    6
Nurse_4    6
Nurse_0    5
Name: Nurse, dtype: int64


In [94]:
# number of extra shifts done by each nurse
print('------------First half of shift------------')
print(sol_z['Nurse'].value_counts())
print('------------Last half hours of shift------------')
print(sol_q['Nurse'].value_counts())

------------First half of shift------------
Nurse_0    4
Nurse_8    2
Nurse_6    2
Nurse_4    2
Nurse_3    1
Nurse_5    1
Nurse_9    1
Name: Nurse, dtype: int64
------------Last half hours of shift------------
Nurse_1    2
Nurse_2    2
Nurse_7    2
Nurse_8    1
Nurse_0    1
Nurse_3    1
Nurse_5    1
Nurse_6    1
Nurse_9    1
Nurse_4    1
Name: Nurse, dtype: int64


### Is the number of nurses $N$ enough to satisfy the demand?

In [95]:
alpha_star_dict =mdl.solution.get_value_dict(alpha)
if max(list(alpha_star_dict.values()))==0:
    print('YES! The number of nurses is enough to satisfy the demand')
else:
    print('NO! The number of nurses is not enough to satisfy the demand')


NO! The number of nurses is not enough to satisfy the demand


### How many hours does each nurse work?

In [96]:
worked_hours = {n:0 for n in nurses}

for i,j in sol_x.iterrows():
    worked_hours[j['Nurse']]+=h[j['Shift']]
worked_hours

{'Nurse_0': 40,
 'Nurse_1': 52,
 'Nurse_2': 52,
 'Nurse_3': 52,
 'Nurse_4': 47,
 'Nurse_5': 52,
 'Nurse_6': 48,
 'Nurse_7': 51,
 'Nurse_8': 48,
 'Nurse_9': 52}

### How many extra hours does each nurse work?

In [98]:
extra_worked_hours = {n:0 for n in nurses}

for i,j in sol_z.iterrows():
    extra_worked_hours[j['Nurse']]+=h[j['Shift']]*0.5
for i,j in sol_q.iterrows():
    extra_worked_hours[j['Nurse']]+=h[j['Shift']]*0.5
extra_worked_hours

{'Nurse_0': 20.0,
 'Nurse_1': 8.0,
 'Nurse_2': 8.0,
 'Nurse_3': 8.0,
 'Nurse_4': 11.5,
 'Nurse_5': 7.5,
 'Nurse_6': 11.5,
 'Nurse_7': 9.0,
 'Nurse_8': 12.0,
 'Nurse_9': 7.5}

### Maximum number of extra hours worked:

In [99]:
max(extra_worked_hours.values())

20.0

### Overall hours worked

In [100]:
overall_worked_hours = worked_hours.copy()
for i,j in extra_worked_hours.items():
    overall_worked_hours[i]+=j

overall_worked_hours

{'Nurse_0': 60.0,
 'Nurse_1': 60.0,
 'Nurse_2': 60.0,
 'Nurse_3': 60.0,
 'Nurse_4': 58.5,
 'Nurse_5': 59.5,
 'Nurse_6': 59.5,
 'Nurse_7': 60.0,
 'Nurse_8': 60.0,
 'Nurse_9': 59.5}

### Visualization tool

Below we provide a tool to check the schedule. 

In [101]:
# remove warning from pandas (in the viz_tool it does what we need)
import warnings
warnings.simplefilter(action='ignore')

In [102]:
def extract_info(nurse,shift,day):
    '''
    interactive function to extract the information required:
    if a value is 'All' then it returns all the values for that specific feature
    '''
    
    global nurses,S,days
    
    
    if nurse == 'All':
        df_tmp = sol_x[(sol_x['Nurse'].isin(nurses))]
    else:
        df_tmp = sol_x[(sol_x['Nurse']==nurse)]

    if shift == 'All':
        df_tmp = df_tmp[(sol_x['Shift'].isin(S))]
    else:
        df_tmp = df_tmp[(sol_x['Shift']==shift)]

    if day == 'All':
        df_tmp = df_tmp[(sol_x['Day'].isin(days))]    
    else:
        df_tmp = df_tmp[(sol_x['Day']==day)]

    print('------------All shift------------')
    print(df_tmp)
    
    if nurse == 'All':
        df_tmp = sol_z[(sol_z['Nurse'].isin(nurses))]
    else:
        df_tmp = sol_z[(sol_z['Nurse']==nurse)]

    if shift == 'All':
        df_tmp = df_tmp[(sol_z['Shift'].isin(S))]
    else:
        df_tmp = df_tmp[(sol_z['Shift']==shift)]

    if day == 'All':
        df_tmp = df_tmp[(sol_z['Day'].isin(days))]    
    else:
        df_tmp = df_tmp[(sol_z['Day']==day)]

    print('------------First half shift------------')

    print(df_tmp)
    
    if nurse == 'All':
        df_tmp = sol_q[(sol_q['Nurse'].isin(nurses))]
    else:
        df_tmp = sol_q[(sol_q['Nurse']==nurse)]

    if shift == 'All':
        df_tmp = df_tmp[(sol_q['Shift'].isin(S))]
    else:
        df_tmp = df_tmp[(sol_q['Shift']==shift)]

    if day == 'All':
        df_tmp = df_tmp[(sol_q['Day'].isin(days))]    
    else:
        df_tmp = df_tmp[(sol_q['Day']==day)]

    print('------------Second half shift------------')
    print(df_tmp)
    

    if shift == 'All':
        df_tmp = sol_alpha[(sol_alpha['Shift'].isin(S))]
    else:
        df_tmp = sol_alpha[(sol_alpha['Shift']==shift)]

    if day == 'All':
        df_tmp = df_tmp[(sol_alpha['Day'].isin(days))]    
    else:
        df_tmp = df_tmp[(sol_alpha['Day']==day)]

    print('------------Non covered shifts------------')
    print(df_tmp)
    


interact(extract_info, nurse = widgets.Dropdown(value="All",placeholder='Type something', options=nurses+['All']),
              shift=widgets.Dropdown(value='All',placeholder='Type something', options=S+['All']),
              day = widgets.Dropdown(value="All",placeholder='Type something', options=days+['All'])
        );



interactive(children=(Dropdown(description='nurse', index=10, options=('Nurse_0', 'Nurse_1', 'Nurse_2', 'Nurse…

## Worst case scenario minimization
In order to get balanced solutions, we should minimize the worst-case scenario by minimizing the number of hours worked by the nurse that works the most and the highest number of nurses missing in the same shift, namely:
\begin{equation}\label{eq: neq_fo}
        \underset{x_{ist},z_{ist}\in\{0,1\}}{\text{min}} \quad \max_i \left\{\sum_{s=1}^3\sum_{t=1}^T  h_s\left( x_{ist} + c z_{ist} +(1-c)  q_{ist}\right)\right\}+\rho_2 \max_{st}\left\{\alpha_{st}\right\}
    \end{equation}
where $\rho_2$ is a big enough penalization term so to ensure that the second term is more important than the first one.

In [103]:
a = mdl.continuous_var(lb=-np.infty)
b = mdl.continuous_var(lb=-np.infty)
mdl.minimize(a+1e4*b)

In [104]:

mdl.add_constraints(a>=mdl.sum(h[s]*(x[i,s,t]+c*z[i,s,t]+(1-c)*q[i,s,t]) for s in S for t in days) for i in nurses)                  
mdl.add_constraints(b>=alpha[s,t] for s in S for t in days);

In [105]:
mdl.solve()

docplex.mp.solution.SolveSolution(obj=10055.5,values={x7:1,x9:1,x10:1,x1..

In [106]:
mdl.report()

* model Scheduling solved with objective = 10055.500
*  KPI: # of shifts not completely covered      = 21.000
*  KPI: Maximum # missing nurses in one shift   = 1.000
*  KPI: Sum of missing nurses in all the shifts = 21.000
*  KPI: Maximum # hours worked                  = 47.000
*  KPI: Minimum # hours worked                  = 38.000


In [109]:
# transform the solution into a dataframe
x_star_dict =mdl.solution.get_value_dict(x)
sol_x = sol_to_pandas(x_star_dict)

z_star_dict =mdl.solution.get_value_dict(z)
sol_z = sol_to_pandas(z_star_dict)

q_star_dict =mdl.solution.get_value_dict(q)
sol_q = sol_to_pandas(q_star_dict)

alpha_star_dict = mdl.solution.get_value_dict(alpha)
sol_alpha = alpha_to_pandas(alpha_star_dict)

In [110]:
print(sol_x['Nurse'].value_counts())


Nurse_8    6
Nurse_1    6
Nurse_2    6
Nurse_5    6
Nurse_6    6
Nurse_9    6
Nurse_4    6
Nurse_7    6
Nurse_0    5
Nurse_3    5
Name: Nurse, dtype: int64


In [111]:
# number of extra shifts done by each nurse
print('------------First half of shift------------')
print(sol_z['Nurse'].value_counts())
print('------------Last half hours of shift------------')
print(sol_q['Nurse'].value_counts())

------------First half of shift------------
Nurse_3    4
Nurse_1    2
Nurse_9    2
Nurse_0    1
Nurse_2    1
Nurse_6    1
Nurse_7    1
Name: Nurse, dtype: int64
------------Last half hours of shift------------
Nurse_0    3
Nurse_8    2
Nurse_5    2
Nurse_4    2
Nurse_2    1
Nurse_6    1
Nurse_7    1
Name: Nurse, dtype: int64


### Is the number of nurses $N$ enough to satisfy the demand?

In [112]:
alpha_star_dict =mdl.solution.get_value_dict(alpha)
if max(list(alpha_star_dict.values()))==0:
    print('YES! The number of nurses is enough to satisfy the demand')
else:
    print('NO! The number of nurses is not enough to satisfy the demand')


NO! The number of nurses is not enough to satisfy the demand


### How many hours does each nurse work?

In [113]:
worked_hours = {n:0 for n in nurses}

for i,j in sol_x.iterrows():
    worked_hours[j['Nurse']]+=h[j['Shift']]
worked_hours

{'Nurse_0': 40,
 'Nurse_1': 46,
 'Nurse_2': 45,
 'Nurse_3': 38,
 'Nurse_4': 46,
 'Nurse_5': 45,
 'Nurse_6': 47,
 'Nurse_7': 45,
 'Nurse_8': 46,
 'Nurse_9': 46}

### Maximum number of hours worked:

In [114]:
max(worked_hours.values())

47

### How many extra hours does each nurse work?

In [115]:
extra_worked_hours = {n:0 for n in nurses}

for i,j in sol_z.iterrows():
    extra_worked_hours[j['Nurse']]+=h[j['Shift']]*0.5
for i,j in sol_q.iterrows():
    extra_worked_hours[j['Nurse']]+=h[j['Shift']]*0.5
extra_worked_hours

{'Nurse_0': 15.5,
 'Nurse_1': 9.0,
 'Nurse_2': 8.5,
 'Nurse_3': 17.5,
 'Nurse_4': 8.0,
 'Nurse_5': 9.0,
 'Nurse_6': 7.5,
 'Nurse_7': 9.0,
 'Nurse_8': 9.0,
 'Nurse_9': 9.0}

### Maximum number of extra hours worked:

In [116]:
max(extra_worked_hours.values())

17.5

### Overall hours worked

In [117]:
overall_worked_hours = worked_hours.copy()
for i,j in extra_worked_hours.items():
    overall_worked_hours[i]+=j

overall_worked_hours

{'Nurse_0': 55.5,
 'Nurse_1': 55.0,
 'Nurse_2': 53.5,
 'Nurse_3': 55.5,
 'Nurse_4': 54.0,
 'Nurse_5': 54.0,
 'Nurse_6': 54.5,
 'Nurse_7': 54.0,
 'Nurse_8': 55.0,
 'Nurse_9': 55.0}